In [1]:
#!/usr/bin/env python
# coding: utf-8

#  Importar Paquetes

# Section[1]:


from CoachAIUtils import *
import re
import json
import IPython.display as ht
import pandas as pd
import numpy as np
import wandb
from ipywidgets import *
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, LSTM, SimpleRNN, Dense, Flatten
from keras import optimizers
from keras import metrics
from keras.callbacks import TensorBoard
from time import time


jupy = True
debug = True

if jupy: get_ipython().magic('matplotlib inline')

#wandb.init(magic=True)
teampar = {
    "trainsetlist": "0,1",
    "trainyearlist": "2007,2008,2009,2010,2011,2012,2013,2014",
    "modeltype": "LSTM",  # @param ["LSTM", "RNN", "Attention", "Red No Recurrente", "Boosting"]
    "eslabones": 3,  # @param {type:"slider", min:2, max:10, step:1}
    "look_back":  3,  # @param {type:"slider", min:2, max:100, step:1}
    "look_fwd": 0, # @param {type:"slider", min:0, max:10, step:1}
    "runs": 1, # @param {type:"slider", min:0, max:5, step:1}
    "overlap":  True,  # @param {type:"boolean"}
    "shuffle":  True,  # @param {type:"boolean"}
    "epochs":  10,  # @param {type:"slider", min:0, max:500, step:10}
    "batch_size":  200,  # @param {type:"slider", min:0, max:10000, step:100}
    "validation_split":  0.2,  # @param {type:"slider", min:0, max:1, step:0.1}
    "optimizador":  "Adam",  # @param ["Adam", "SGD", "RMS", "Adagrad", "Adadelta", "Adamax", "Nadam"]
    "lr":  0.1,  # @param {type:"slider", min:0, max:1, step:0.001}
    "decay":  0.1,  # @param {type:"slider", min:0, max:1, step:0.001}
    "dropout":  0,  # @param {type:"slider", min:0, max:1, step:0.001}
    "recurrent_dropout":  0,  # @param {type:"slider", min:0, max:1, step:0.001}
    "determinista":  False}

savefich = ""
loadfichteam = ""
loadfichport = "21Oct19"
dire=".\\"
yearstrainlist = ""
easetstrain = []
yearstrain = []
alfas=[]

con = conectamysql("mysql+pymysql://{}:{}@{}/{}".format(user, passwd, host, db))
dfhead = pd.read_sql("select distinct EASetName from easet", con)
df = pd.read_sql("select distinct ﻿EASetName, Modifications from easettrades", con)
easetlist = df["﻿EASetName"].unique()


Using TensorFlow backend.


In [2]:
def muestrasets():
    print("Sets Disponibles:")

    i = 0
    s = "<table><tr>"
    for st in easetlist:
        s += "<td>{}</td><td>{}</td>".format(i, nombrebueno(df, st))
        if i % 4 == 0:
            s += "</tr><tr>"
        i += 1
    s += "</tr></table>"
    if jupy:
        ht.display(ht.HTML(s))
    else:
        print(s)

    faltansets = False
    for st in dfhead["EASetName"].unique():
        if len(df[df["﻿EASetName"] == st]) == 0:
            if faltansets is False:
                print("Sets Parseados pero no disponibles:")
                faltansets = True
            print(st)

    if faltansets is False:
        print("Ninguno")
    return

def create_dataset(dataset, pcol, plback=2, plfwd=0):
    datax, datay, desctrain = [], [], []
    cols = {"easet": 0, "ticket": 1, "apertura": 4, "cierre": 5, "lote": 8, "pl": 28, "acc": 29}
    salto = 1 if setpar["overlap"] else setpar["eslabones"]
    lookback = plback * salto
    lookahead = plfwd * salto

    tot = 1
    for i in range(len(dataset)):
        for j in range(0, len(dataset[i]) - lookback - lookahead, salto):
            a = dataset[i, range(j, j + lookback, salto), pcol]
            b = dataset[i, j + lookback + lookahead, pcol]
            if b == 0.0 :
                continue
            c = dataset[i, j + lookback + lookahead, (cols["easet"], cols["easet"], cols["ticket"], cols["apertura"], cols["cierre"],
                                                      cols["lote"], cols["pl"], cols["acc"])]
            c[6] += sum(dataset[i, j + lookback + lookahead - salto:j + lookback + lookahead, cols["pl"]])
            c[3] = fechabuena(c[3])
            ano = c[3][:4]
            c[4] = fechabuena(c[4])
            c[1] = nombrebueno(df, c[1])
            desc = c[1] + "/" + ano

            if (c[0] in easetstrain) and (ano in yearstrain):
                datax.append(a)
                datay.append(b)
                if desc not in desctrain:
                    desctrain.append(desc)
            tot += 1

    if debug:
        print("Sets con los que Entrenará:", desctrain)
    return np.array(datax), np.array(datay)


def cargaparteam(ptrainyearlist,
        pmodeltype, peslabones, plook_back, plook_fwd, pruns, poverlap, pshuffle, pepochs, pbatch_size, pvalidation_split, poptimizador, plr, pdecay, pdropout,
        precurrent_dropout, ploadfichteam, ploadfichport):
    global teampar, loadfich, alfas

    teampar = {"trainyearlist": ptrainyearlist, "modeltype": pmodeltype, "eslabones": peslabones, "look_back": plook_back, "look_fwd": plook_fwd, "runs": pruns, "overlap": poverlap, "shuffle": pshuffle,
        "epochs": pepochs, "batch_size": pbatch_size, "validation_split": pvalidation_split,
        "optimizador": poptimizador, "lr": plr, "decay": pdecay, "dropout": pdropout, "recurrent_dropout": precurrent_dropout}
    loadfichteam = ploadfichteam
    if loadfichteam != "":
        teampar = json.load(open(dire + "teampar-" + ploadfichteam + ".json"))
        print ("Cargado "+ ploadfichteam)

    portpred = pd.read_csv(dire + "portdat-" + ploadfichport + ".csv", sep=";", index_col=0)
    for y in ptrainyearlist.split(","):
        yearstrain.append(y)

    alfas = portpred.loc["NomAjuste"]
    return "Cargado"


def modela(px_train, py_train, pmodeltype="LSTM", pretseq=False):
    if debug:
        print(px_train.shape, py_train.shape)

    lmodel = Sequential()

    # Recurrent layer
    if pmodeltype == "LSTM":
        lmodel.add(
            LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq, dropout=setpar["dropout"],
                 recurrent_dropout=setpar["recurrent_dropout"]))
    elif pmodeltype == "RNN":
        lmodel.add(SimpleRNN(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq,
                             dropout=dropout, recurrent_dropout=recurrent_dropout))
    elif pmodeltype == "Attention":
        lmodel = keras.models.Sequential()
        lmodel.add(keras.layers.Embedding(input_dim=10000, output_dim=300, mask_zero=True))
        lmodel.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=True)))
        lmodel.add(SeqSelfAttention(attention_activation='sigmoid'))
        lmodel.add(keras.layers.Dense(units=5))
        lmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'], )
        lmodel.summary()
    elif pmodeltype == "Red No Recurrente":
        lmodel.add(Dense(20, input_shape=(px_train.shape[1],), activation='sigmoid'))
        lmodel.add(Dense(5, activation='sigmoid'))
        # Error when checking target: expected activation_1 to have 3 dimensions, but got array with shape (3243, 5)
    elif pmodeltype == "Boosting":
        print("Boosting en desarrollo")
        exit(-1)
        # lmodel.add(LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=True, dropout=dropout,
        #                recurrent_dropout=recurrent_dropout))
        # lmodel.add(SeqSelfAttention(5, attention_activation='sigmoid'))
    elif pmodeltype == "Logistic Regression":
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(px_train, py_train)
        #clf.predict([:2, :])
        #clf.predict_proba(X[:2, :])
        #clf.score(X, y)

    lmodel.add(Activation('softmax'))
    # Compile the model
    if setpar["optimizador"] == "SGD":
        opt = optimizers.SGD(lr=lr, momentum=0.0, decay=decay, nesterov=False)
    elif setpar["optimizador"] == "Adam":
        opt = optimizers.Adam(lr=setpar["lr"], beta_1=0.9, beta_2=0.999, epsilon=None, decay=setpar["decay"], amsgrad=False)
    elif setpar["optimizador"] == "RMS":
        opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adagrad":
        opt = optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adadelta":
        opt = optimizers.Adadelta(lr=lr, rho=0.95, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Adamax":
        opt = optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay)
    elif setpar["optimizador"] == "Nadam":
        optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=decay)

    lmodel.compile(optimizer=opt, loss='binary_crossentropy', metrics=[metrics.categorical_accuracy])

    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    history = lmodel.fit(px_train, y_train, verbose=(0 + debug * 2), validation_split=setpar["validation_split"], epochs=setpar["epochs"],
                         batch_size=setpar["batch_size"], shuffle=setpar["shuffle"], callbacks=[tensorboard])
    print(lmodel.summary())
    return lmodel


# Datos de Set

## Selección de Datos
##  ___________________________
Sets a incluir en training:ptrainsetlist

Años a incluir en training:ptrainyearlist

Sets a incluir en test:ptestsetlist

Años a incluir en test:ptestyearlist
 
## Parámetros Set
## ______________
pmodeltype: Modelo

peslabones: Tipo de eslabon (eg: 5 mira los 5 ultimos trades)

plook_back: Longitud de la cadena (eg: 10 entrena con los ultimos 10 eslabones)

plook_fwd: Lag de la predicción (0 intenta predecir siguiente eslabon, 1 dos eslabones despues...)

pruns: Numero de veces que entrenamos

Eslabones solapados o secuenciales:poverlap

Si marcas multiyear, habra un grafico por set y no por año: pmultiyear

Nuevo lotaje:plota, plotb, plotc, plotd

Tamaño Gráficos:pgrafancho, pgrafalto
 
## Parametros de los modelos:
## __________________________
Número de pasadas:pepochs

Número de elementos en cada pasada:pbatch_size

% de los datos que se usan para validación:pvalidation_split

Optimizador: poptimizador

Learning Rate: Tamaño de la actualizacion en gradient descent:plr

Reduccion progresiva del learning rate:pdecay

Celdas a desactivar cada pasada del estado oculto para prevenir overfitting:pdropout

Celdas a desactivar de la "memoria" para prevenir overfitting:precurrent_dropout
 

In [4]:
interact_manual(cargaparteam,
                ptrainyearlist= teampar["trainyearlist"],
                pmodeltype = teampar["modeltype"],  # @param ["LSTM", "RNN", "Attention", "Red No Recurrente", "Boosting"]
                peslabones = teampar["eslabones"],  # @param {type:"slider", min:2, max:10, step:1}
                plook_back = teampar["look_back"],  # @param {type:"slider", min:2, max:100, step:1}
                plook_fwd = teampar["look_fwd"],  # @param {type:"slider", min:0, max:10, step:1}
                pruns = teampar["runs"],  # @param {type:"slider", min:0, max:10, step:1}
                poverlap = teampar["overlap"],  # @param {type:"boolean"}
                pshuffle = teampar["shuffle"],  # @param {type:"boolean"}
                pepochs = teampar["epochs"],  # @param {type:"slider", min:0, max:500, step:10}
                pbatch_size = teampar["batch_size"],  # @param {type:"slider", min:0, max:10000, step:100}
                pvalidation_split = teampar["validation_split"],  # @param {type:"slider", min:0, max:1, step:0.1}
                poptimizador = teampar["optimizador"],  # @param ["Adam", "SGD", "RMS", "Adagrad", "Adadelta", "Adamax", "Nadam"]
                plr = teampar["lr"],  # @param {type:"slider", min:0, max:1, step:0.001}
                pdecay = teampar["decay"],  # @param {type:"slider", min:0, max:1, step:0.001}
                pdropout = teampar["dropout"],  # @param {type:"slider", min:0, max:1, step:0.001}
                precurrent_dropout = teampar["recurrent_dropout"],  # @param {type:"slider", min:0, max:1, step:0.001}
                ploadfichteam = loadfichteam,
                ploadfichport = loadfichport)

print("Configuraciones de Sets")
for parfich in os.listdir(dire):
    if re.match("teampar.*.json", parfich):
        print(parfich[7:-5])


interactive(children=(Text(value='2007,2008,2009,2010,2011,2012,2013,2014', description='ptrainyearlist'), Tex…

Configuraciones de Sets
-21Oct19


In [5]:
def create_dataset_alfas(dataset, plback_alfas=2, plfwd_alfas=0, ptestsplit=200):
    datax, datay = [], []
    lookback = plback_alfas
    lookahead = plfwd_alfas

    tot = 1
    for j in range(0, len(dataset) - lookback - lookahead):
        a = dataset[j: j + lookback]
        b = dataset[j + lookback + lookahead]
        if b == 0.0:
            continue
        if j < ptestsplit:
            datax.append(a)
            datay.append(b)
        tot += 1

    return np.array(datax), np.array(datay)

def modela_alfas(px_train, py_train, pmodeltype="LSTM", pretseq=False):
    mapeo = {"@": 0, "Alfa": 1, "Beta": 2, "Gamma":3, "Delta": 4}
    for x in np.nditer(px_train, flags=['refs_ok'], op_flags=['readwrite']):
        x[...] = mapeo[str(x)]
    px_train = to_categorical(px_train, num_classes=5)
    for x in np.nditer(py_train, flags=['refs_ok'], op_flags=['readwrite']):
        x[...] = mapeo[str(x)]
    py_train = to_categorical(py_train, num_classes=5)
    if debug:
        print(px_train.shape, py_train.shape)

    lmodel = Sequential()

    # Recurrent layer
    if pmodeltype == "LSTM":
        lmodel.add(
            LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq, dropout=teampar["dropout"],
                 recurrent_dropout=teampar["recurrent_dropout"]))
    elif pmodeltype == "RNN":
        lmodel.add(SimpleRNN(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=pretseq,
                             dropout=teampar["dropout"], recurrent_dropout=teampar["recurrent_dropout"]))
    elif pmodeltype == "Attention":
        lmodel = keras.models.Sequential()
        lmodel.add(keras.layers.Embedding(input_dim=10000, output_dim=300, mask_zero=True))
        lmodel.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=True)))
        lmodel.add(SeqSelfAttention(attention_activation='sigmoid'))
        lmodel.add(keras.layers.Dense(units=5))
        lmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'], )
        lmodel.summary()
    elif pmodeltype == "Red No Recurrente":
        lmodel.add(Dense(20, input_shape=(px_train.shape[1],), activation='sigmoid'))
        lmodel.add(Dense(5, activation='sigmoid'))
        # Error when checking target: expected activation_1 to have 3 dimensions, but got array with shape (3243, 5)
    elif pmodeltype == "Boosting":
        print("Boosting en desarrollo")
        exit(-1)
        # lmodel.add(LSTM(5, input_shape=(px_train.shape[1], px_train.shape[2]), return_sequences=True, dropout=dropout,
        #                recurrent_dropout=recurrent_dropout))
        # lmodel.add(SeqSelfAttention(5, attention_activation='sigmoid'))
    elif pmodeltype == "Logistic Regression":
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(px_train, py_train)
        #clf.predict([:2, :])
        #clf.predict_proba(X[:2, :])
        #clf.score(X, y)

    lmodel.add(Activation('softmax'))
    # Compile the model
    if teampar["optimizador"] == "SGD":
        opt = optimizers.SGD(lr=teampar["lr"], momentum=0.0, decay=teampar["decay"], nesterov=False)
    elif teampar["optimizador"] == "Adam":
        opt = optimizers.Adam(lr=teampar["lr"], beta_1=0.9, beta_2=0.999, epsilon=None, decay=teampar["decay"], amsgrad=False)
    elif teampar["optimizador"] == "RMS":
        opt = optimizers.RMSprop(lr=teampar["lr"], rho=0.9, epsilon=None, decay=teampar["decay"])
    elif teampar["optimizador"] == "Adagrad":
        opt = optimizers.Adagrad(lr=teampar["lr"], epsilon=None, decay=teampar["decay"])
    elif teampar["optimizador"] == "Adadelta":
        opt = optimizers.Adadelta(lr=teampar["lr"], rho=0.95, epsilon=None, decay=teampar["decay"])
    elif teampar["optimizador"] == "Adamax":
        opt = optimizers.Adamax(lr=teampar["lr"], beta_1=0.9, beta_2=0.999, epsilon=None, decay=teampar["decay"])
    elif teampar["optimizador"] == "Nadam":
        optimizers.Nadam(lr=teampar["lr"], beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=teampar["decay"])

    lmodel.compile(optimizer=opt, loss='binary_crossentropy', metrics=[metrics.categorical_accuracy])

    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    history = lmodel.fit(px_train, py_train, verbose=(0 + debug * 2), validation_split=teampar["validation_split"], epochs=teampar["epochs"],
                         batch_size=teampar["batch_size"], shuffle=teampar["shuffle"], callbacks=[tensorboard])
    print(lmodel.summary())
    return lmodel

def creamodel_alfas(palfas):
    # Slicear data
    X_train, y_train = create_dataset_alfas(palfas, plback_alfas=teampar["look_back"], plfwd_alfas=teampar["look_fwd"])

    if len(y_train) == 0:
        print("No hay datos de entrenamiento, revisa sets y años pls")
        exit(0)

    # Entrenar Alfas
    model = modela_alfas(X_train, y_train, pmodeltype=teampar["modeltype"])
    return(model)

## Si no sale nada, dar a Run All Above
## Despues de cargar una configuracion, dar al play en la celda de parametros para refrescar

if not jupy:
    cargaparteam(yearstrainlist,
                 teampar["modeltype"], teampar["eslabones"], teampar["look_back"], teampar["look_fwd"], teampar["runs"], teampar["overlap"], teampar["shuffle"], teampar["epochs"], teampar["batch_size"],
                 teampar["validation_split"], teampar["optimizador"], teampar["lr"], teampar["decay"], teampar["dropout"], teampar["recurrent_dropout"], loadfichteam, loadfichport)

model = creamodel_alfas(alfas)

def grabafich(psavefich):
    global savefich

    savefich = psavefich
    if psavefich != "":
        savejson(teampar, dire + "teampar-" + psavefich)
        model.save(dire + "team-" + psavefich + ".h5")
        print("Grabado " + psavefich)

    return "Grabado"


(200, 3, 5) (200, 5)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 160 samples, validate on 40 samples
Epoch 1/10
 - 1s - loss: 0.4988 - categorical_accuracy: 0.0063 - val_loss: 0.4462 - val_categorical_accuracy: 1.0000
Epoch 2/10
 - 0s - loss: 0.4460 - categorical_accuracy: 0.9937 - val_loss: 0.4106 - val_categorical_accuracy: 1.0000
Epoch 3/10
 - 0s - loss: 0.4108 - categorical_accuracy: 0.9937 - val_loss: 0.3728 - val_categorical_accuracy: 1.0000
Epoch 4/10
 - 0s - loss: 0.3733 - categorical_accuracy: 0.9937 - val_loss: 0.3413 - val_categorical_accuracy: 1.0000
Epoch 5/10
 - 0s - loss: 0.3421 - categorical_accuracy: 0.9937 - val_loss: 0.3170 - val_categorical_accuracy: 1.0000
Epoch 6/10
 - 0s - loss: 0.3180 - categorical_accuracy: 0.9937 - val_loss: 0.2947 - val_categorical_accuracy: 1.0000
Epoch 7/10
 - 0s - loss: 0.2959 - categorical_accuracy: 0.9937 - val_loss: 0.2722 - val_categorical_accuracy: 1.

## Si no sale nada, dar a Run All Above
## Despues de cargar una configuracion, dar al play en la celda de parametros para refrescar

In [6]:
interact_manual(grabafich,
                psavefich= savefich)


interactive(children=(Text(value='', description='psavefich'), Button(description='Run Interact', style=Button…

<function __main__.grabafich>